## 3.1 Dimension Reduction 

* 為容易理解高維資料的分佈情況及降低後續特徵提取演算量，最常用的方式就是將資料「降維(Dimensionality Reduction)」
到二維或三維空間再進行觀察，亦可看做是將資料從高維度重新投影(Projection)至低維度空間，包括
    * 線性 <br>
        A. 主成份分析(Principle Component Analysis, PCA) <br>
        線性區別分析(Linear Discriminant Analysis, LDA) <br>
        B. 非負矩陣分解(Non-negative Matrix Factorization, NMF) <br>
    * 非線性<br>
        C. t-分佈隨機鄰近嵌入(t-distributed Stochastic Neighbor Embedding, t-SNE) <br>
        D. 均勻流形逼近及投影(Uniform Manifold Approximation and Projection, UMAP) <br>

<hr style="border:2px solid green">

### A. 主成分析 (Principal Component Analysis, PCA)
PCA is fundamentally a dimensionality reduction algorithm, but it can also be useful as a tool for visualization, for noise filtering, for feature extraction and engineering, and much more.
https://www.youtube.com/watch?v=g-Hb26agBFg

https://miro.medium.com/v2/resize:fit:1100/format:webp/0*S4yAgwWKpy0ud5-q

<div>
<img src="PCA.bin" width="800"/>
</div>

* 主成分分析步驟
    * Step 1: Find mean and normalization (標準化d維原”數據集”)。
    * Step 2: Derive the covariance matrix (建立共變異數矩陣)。
    * Step 3: Find the eigenvalues and eigenvectors (特徵值與特徵向量)。
    * Step 4: Select principle componments (選取k個最大特徵值相對應k個的特徵向量，其中k即為新特徵子空間的維數)。
    * Step 5: Transpose to dimension reduced data (使用排序最上面的k個的特徵向量，建立投影矩陣W以轉換原本d維的原數據至新的k維特徵子空間)。

### PCA Implemenration

http://localhost:8888/notebooks/Machine%20Learning%20with%20Application/3.%20Unsupervised%20Learning/PCA_Implementatoin.ipynb

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()

fig, axes = plt.subplots(2, 5, figsize=(8, 5),
                         subplot_kw={'xticks':(), 'yticks': ()})
for ax, img in zip(axes.ravel(), digits.images):
    ax.imshow(img)

In [ ]:
len(digits.target)

In [ ]:
from sklearn.decomposition import PCA
# build a PCA model
pca = PCA(n_components=2)
pca.fit(digits.data)

# transform the digits data onto the first two principal components
digits_pca = pca.transform(digits.data)

In [ ]:
colors = ["#476A2A", "#7851B8", "#BD3430", "#4A2D4E", "#875525",
          "#A83683", "#4E655E", "#853541", "#3A3120", "#535D8E"]
plt.figure(figsize=(5, 5))
plt.xlim(digits_pca[:, 0].min(), digits_pca[:, 0].max())
plt.ylim(digits_pca[:, 1].min(), digits_pca[:, 1].max())

for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_pca[i, 0], digits_pca[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("First principal component")
plt.ylabel("Second principal component")

#### Application: Eigenfaces for feature extraction

In [ ]:
from sklearn.datasets import fetch_lfw_people
people = fetch_lfw_people(min_faces_per_person=20, resize=0.7)
 # min_faces_per_person: The extracted dataset will only retain pictures of people that have at least min_faces_per_person different pictures
image_shape = people.images[0].shape

In [ ]:
print(f"people.images.shape: {people.images.shape}")
print(f"Number of classes: {len(people.target_names)}")
print(f'1st people name: {people.target_names[people.target[1]]}')

In [ ]:
fig, axes = plt.subplots(2, 5, figsize=(8, 5),
                         subplot_kw={'xticks': (), 'yticks': ()})

for target, image, ax in zip(people.target, people.images, axes.ravel()):
    ax.imshow(image)
    ax.set_title(people.target_names[target])

In [ ]:
# count how often each target appears
counts = np.bincount(people.target)
# print counts next to target names:
for i, (count, name) in enumerate(zip(counts, people.target_names)):
    print("{0:25} {1:3}".format(name, count), end='   ')
    if (i + 1) % 3 == 0:
        print()

In [ ]:
np.unique(people.target) # take the unique index of the images

In [ ]:
mask = np.zeros(people.target.shape, dtype = np.bool_)
len(mask)  # sample sizes

In [ ]:
np.where(people.target == 0)

In [ ]:
for target in np.unique(people.target):
    mask[np.where(people.target == target)[0][:50]] = 1
    
X_people = people.data[mask]
y_people = people.target[mask]

# scale the grey-scale values to be between 0 and 1
# instead of 0 and 255 for better numeric stability:
X_people = X_people / 255.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# split the data in training and test set
people_X_train, people_X_test, people_y_train, people_y_test = train_test_split(
    X_people, y_people, stratify=y_people, random_state=0)
# build a KNeighborsClassifier with using one neighbor:
people_knn = KNeighborsClassifier(n_neighbors=50)
people_knn.fit(people_X_train, people_y_train)
print("Test set score of 1-nn: {:.2f}".format(people_knn.score(people_X_test, people_y_test)))

In [ ]:
people_pca = PCA(n_components=100, whiten=True, random_state=0).fit(people_X_train)
people_X_train_pca = people_pca.transform(people_X_train)
people_X_test_pca = people_pca.transform(people_X_test)

print("X_train_pca.shape: {}".format(people_X_train_pca.shape))

In [ ]:
print("pca.components_.shape: {}".format(people_pca.components_.shape))

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(8, 5),
                         subplot_kw={'xticks': (), 'yticks': ()})
for i, (component, ax) in enumerate(zip(people_pca.components_, axes.ravel())):
    ax.imshow(component.reshape(image_shape),
              cmap='viridis')
    ax.set_title("{}. component".format((i + 1)))

由於PCA主要目的是找出資料集中最重要的特徵維度及使其有最大分佈，並不關心資料原始排列結構及相互關連性，所以當資料維度變大時，只靠一個或兩個軸向的表示就變得難以正確的進行資料分類。

* PCA is widely used for dimensionality reduction and is a linear transformation in the sense that the new dimensions it produces are a linear combination of the original dimensions. 
* It focusses more on the dissimilar points because it tries to maximize the distance between the points that are far away in high dimensions.
* However, this comes at the cost of assuming a linear global structure of the dataset. 
* So, for non-linear datasets where large distances are usually less indicative of the actual structure of the dataset (check the image below), it makes more sense to use small distances between points, i.e. consider neighboring points.

#### LDA是PCA延伸的一種方法，
- PCA目標是希望找到投影軸讓資料投影下去後分散量最大化，但PCA不需要知道資料的類別。
- LDA也是希望資料投影下去後分散量最大，但不同的是這個分散量是希望「不同類別之間的分散量」越大越好。
- LDA和PCA差異: PCA是無監督式(unsupervised learning)方法，LDA多了這四個字(「不同類別」)是一種監督式(supervised learning)方法
    
https://chih-sheng-huang821.medium.com/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-%E9%99%8D%E7%B6%AD-dimension-reduction-%E7%B7%9A%E6%80%A7%E5%8D%80%E5%88%A5%E5%88%86%E6%9E%90-linear-discriminant-analysis-d4c40c4cf937

<hr style="border:2px solid green">

### B. 非負矩陣分解 (Non-Negative Matrix Factorization, NMF)

The why and how of nonnegative matrix factorization: https://blog.acolyer.org/2019/02/18/the-why-and-how-of-nonnegative-matrix-factorization/ <br>
NMF was first introduced by Paatero and Tapper in 1994 and popularised in an article by Lee and Seung in 1999. Since then, the number of publications referencing the technique has grown rapidly.

* MF approximates a matrix $\mathbf{X}$ with a low-rank matrix approximation such that $\mathbf{X}\approx \mathbf{WH}$.
* NMF has become so popular because of its ability to extract sparse and easily interpretable factors automatically.

<div>
<img src="https://blog.acolyer.org/wp-content/uploads/2019/02/nmf-fig-1.jpeg?w=640" width="1000"/>
</div>

Assume there exists a dataset $\mathbf{X}$, we can seperate $\mathbf{X} $ into the dictionary of elementary recurring features of the data $\mathbf{W}$ and the activations of the different features $\mathbf{H}$

\begin{equation}
\begin{split}
    \mathbf{X} \approx \mathbf{WH}, \mathbf{W} \geq 0 , \mathbf{H} \geq 0
\end{split}. \tag{B.1}
\end{equation} 

where $k$ is the number of latent features, such as $\mathbf{X} \in \boldsymbol{R}^{p \times n}; \mathbf{W} \in \boldsymbol{R}^{p \times r}; \mathbf{H} \in \boldsymbol{R}^{r \times n}$

##### Step 1: Initialize the basis matrix $\mathbf{H}$ 
    
##### Step 2: Update $\mathbf{H}$ by gradient descent
The Euclidean distance $ ||\mathbf{X} - \mathbf{W H}||$ is non increasing under the update rules. The Euclidean distance is invariant under these updates if and only if $\mathbf{W}$ and $\mathbf{H}$ are at a stationary point of the distance. Hence, we have

\begin{equation}
\begin{split}
    \mathbf{H}_{j}^{(\tau+1)} = \mathbf{H}_{j}^{(\tau)} + \eta [(\mathbf{W}^T \mathbf{X})_{j} - (\mathbf{W}^T \mathbf{WH})_{j}]
\end{split}. \tag{B.2}
\end{equation} 
where <br>
$\eta = \frac{\mathbf{H}}{\mathbf{W}^T\mathbf{WH}}$: learning rate <br> 
$\tau$: the iteration number.<br>
$j$: the sample number $j \in \boldsymbol{N}^{+}$ and $\leq n$

##### Step 3: Update the coefficient matrix $\mathbf{W}$
\begin{equation}
\begin{split}
    \mathbf{W}_{j}^{(\tau+1)} =  \mathbf{W}_{j}^{(\tau)} + \eta [(\mathbf{X} \mathbf{H}^T)_{j} - (\mathbf{WH} \mathbf{H}^T)_{j}]
\end{split}. \tag{B.3}
\end{equation} 

##### Step 4: repeat until 

\begin{equation}
\begin{split}
    \lVert \sum_{i=1}^{n} \mathbf{X}_i - (\mathbf{WH})_i  \lVert \approx 0 
\end{split}. \tag{B.4}
\end{equation} 

The objective function of sklearn.decomposition.NMF is:
    
\begin{align}
\begin{aligned}
L(\mathbf{W}, \mathbf{H}) &= 0.5 * ||\mathbf{X} - \mathbf{WH}||_{loss}^2\\&+ \alpha_W * l1_{ratio} * n_{features} * ||vec(\mathbf{W})||_1\\&+ \alpha_H * l1_{ratio} * n_{samples} * ||vec(\mathbf{H})||_1\\&+ 0.5 * \alpha_W * (1 - l1_{ratio}) * n_{features} * ||\mathbf{W}||_{Fro}^2\\&+ 0.5 * \alpha_H * (1 - l1_{ratio}) * n_{samples} * ||\mathbf{H}||_{Fro}^2
\end{aligned}. \tag{B.5}
\end{align}

#### Note: 
- PCA: generate a face from multiple reduced faces
- NMF: synthesis a face based on key basis (features: such as eye from A, node from B,...), in other words, $\mathbf{H}$ is a feature matrix.

#### B.1 Applying NMF to synthetic data

In [ ]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=15, random_state=0, max_iter=20000)
nmf.fit(people_X_train)
people_X_train_nmf = nmf.transform(people_X_train)
people_X_test_nmf = nmf.transform(people_X_test)

In [ ]:
fig, axes = plt.subplots(3, 5, figsize=(8, 5),
                         subplot_kw={'xticks': (), 'yticks': ()})
for i, (component, ax) in enumerate(zip(nmf.components_, axes.ravel())):
    ax.imshow(component.reshape(image_shape))
    ax.set_title("Comp. #{}".format(i))

In [ ]:
compn = 3
# sort by 3rd component, plot first 10 images
inds = np.argsort(people_X_test_nmf[:, compn])[::-1]
fig, axes = plt.subplots(2, 5, figsize=(8, 5),
                         subplot_kw={'xticks': (), 'yticks': ()})
fig.suptitle("Large component 3")
for i, (ind, ax) in enumerate(zip(inds, axes.ravel())):
    ax.imshow(people_X_test[ind].reshape(image_shape))

https://medium.com/@zahrahafida.benslimane/audio-source-separation-using-non-negative-matrix-factorization-nmf-a8b204490c7d

<hr style="border:2px solid green">

### C. t-SNE (t-Distributed Stochastic Neighbor Embedding)
https://builtin.com/data-science/tsne-python <br>
Understanding t-SNE: https://newmortalbeing.medium.com/understanding-t-sne-d869ece441a5

<hr style="border:1px solid grey">

#### Naive_Bayes_Implementation: 
http://localhost:8888/notebooks/Machine%20Learning%20with%20Application/3.%20Unsupervised%20Learning/Naive_Bayes_Implementation.ipynb#

<hr style="border:1px solid grey">

* t-SNE is an unsupervised, non-parametric method for dimensionality reduction developed by Laurens van der Maaten and Geoffrey Hinton in 2008.
* ‘Non-parametric’ because it doesn’t construct an explicit function that maps high dimensional points to a low dimensional space. 
* Instead, it just optimizes low dimensional positions of the data points directly.

- t-SNE 主要是將高維的數據用高斯分佈的機率密度函數近似，而低維數據的部分使用 t 分佈的方式來近似，https://www.jmp.com/zh_tw/statistics-knowledge-portal/t-test/t-distribution.html
- 使用 KL 距離計算相似度，https://machinelearningmastery.com/divergence-between-probability-distributions/
- 最後再以梯度下降（或隨機梯度下降）求最佳解 。

#### Step 1. Define similarities between points<br> 

A Multivariate Gaussian is centered at each point $x_i$ of the dataset. For every other point $x_j$ of the dataset, we calculate the similarity between $x_i$ and $x_j$ as the conditional probability $p_{j|i}$.

\begin{equation}
\begin{split}
{p_ {j \mid i} = \frac{\exp(- \mid  \mid  x_i -x_j  \mid  \mid  ^2 / (2 \sigma^2_i ))} {\sum_{k \neq i} \exp(- \mid  \mid  x_i - x_k  \mid  \mid  ^2 / (2 \sigma^2_i))}}
\end{split}. \tag{C.1}
\end{equation}

where $\sigma_i$ is the variance of Gaussian distributed with mean $x_i$, and $p_{x∣x} =0$.

<div style="text-align: center;">
<img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*fYhwN3wKr3NwDSJFePqwqQ.png" width="300"/>
</div>

This probability is proportional to the probability density of the Gaussian kernel centered at $x_i$. 
So, for any $x_j$( for example, the yellow data point) that is close to $x_i$, this probability would be high and for any far away $x_j$ (the red point), it is going to be very small but never zero.

#### Step 2. Low dimension similarities
* SNE arranges all the data points on the required low dimensional space by randomly sampling from a Gaussian that has a very small variance and is centered around the origin. 
* Note that we are not trying to preserve distances here since it is impossible to preserve all high dimensional distances between points in the low dimensional space. 
* Instead, we try to preserve the order/rank of distances between points in SNE.

The similarity between points in the low dimensional space is given by the conditional probability qⱼ|ᵢ .

\begin{equation}
\begin{split}
{q_ {j \mid i} = \frac{\exp(- \mid  \mid  y_i -y_j  \mid  \mid  ^2 )} {\sum_{k \neq i} \exp(- \mid  \mid  y_i - y_k  \mid  \mid  ^2 )}}
\end{split}. \tag{C.2}
\end{equation}

It calculates the similarities for all pair of points to produce the embedding similarity matrix.

#### Step 3. Loss function and optimization

* We want the low dimensional mappings {$y_1$, $y_2$,…., $y_n$} to have similar probability distributions as their high dimensional counterparts so as to preserve the relationship between the points in the embedding.

* For this, the sum of KL divergences between the pairwise similarities in the high dimension and low dimensional spaces is used as the loss function. 

\begin{equation}
\begin{split}
J = \sum_i \text{KL}(P_i \Vert Q_i) = \sum_i\sum_j p_{j \vert i} \log \frac{p_{j \vert i}}{q_{j \vert i}}
\end{split}. \tag{C.3}
\end{equation}

* KL Divergence function is asymmetric by nature, meaning KL($P_i$ || $Q_i$) ≠ KL( $Q_i$ || $P_i$).<br>
And this property of KL Divergence is leveraged by the algorithm to focus on preserving the local structure of the data. 

* If $p_{j \vert i}$ is large and $q_{j \vert i}$  is small, there is a large cost/penalty for representing nearby points in the high dimensional space by widely separated points in the low dimensional map. This explains why the technique is good at preserving the local structure.
* However, if$p_{j \vert i}$  is small, any $q_{j \vert i}$  will have almost similar small penalties.
* In other words, far away points in high dimensional space can be be placed just anywhere on the low dimensional map.

After initializing the low dimensional mappings and calculating the similarities, the loss function is optimized using Gradient Descent to arrive at the final embedding.

\begin{equation}
\begin{split}
\frac{\partial J}{\partial y_i} = 2 \sum_{j \neq i} (p_{j \vert i} - q_{j \vert i} + p_{i \vert j} - q_{i \vert j})(y_i - y_j)
\end{split}. \tag{C.4}
\end{equation}

Gradient Update in each iteration :
    
\begin{equation}
\begin{split}
y^{(k)} = y^{(k-1)} + \eta \frac{\partial J}{\partial y_i} + \alpha(k) (y^{(k-1)}-y^{(k-2)})
\end{split}. \tag{C.5}
\end{equation}
                                                                                       
where $\alpha(k)$ is monentum at iteration k, $y^{(k)}$ is the soluation at iteration k, and $\eta$ is learning rate.

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(random_state=42)
# use fit_transform instead of fit, as TSNE has no transform method
digits_tsne = tsne.fit_transform(digits.data)

In [ ]:
plt.figure(figsize=(5, 5))
plt.xlim(digits_tsne[:, 0].min(), digits_tsne[:, 0].max() + 1)
plt.ylim(digits_tsne[:, 1].min(), digits_tsne[:, 1].max() + 1)
for i in range(len(digits.data)):
    # actually plot the digits as text instead of using scatter
    plt.text(digits_tsne[i, 0], digits_tsne[i, 1], str(digits.target[i]),
             color = colors[digits.target[i]],
             fontdict={'weight': 'bold', 'size': 9})
plt.xlabel("t-SNE feature 0")
plt.ylabel("t-SNE feature 1")

* Problems with SNE and the modifications made
* Cost function of SNE is difficult to optimize
* Crowding Problem

* One good practice is to first use PCA to reduce the number of dimensions (if d is large) and then use t-SNE.

* Many ways have been suggested over the years to speed up the computation of the attractive and repulsive forces.

#### D. 均勻流形逼近及投影(Uniform Manifold Approximation and Projection, UMAP)
UMAP is a manifold learning technique that aims to reduce the dimensionality of data while preserving its topological structure. <br>
It is particularly useful for visualizing high-dimensional datasets in a low-dimensional space, typically two or three dimensions. <br>
UMAP is often compared to t-SNE (t-distributed Stochastic Neighbor Embedding) due to its similar application in data visualization, but it offers several advantages, including better preservation of global data structure and faster computation times.

https://umap-learn.readthedocs.io/en/latest/basic_usage.html

https://www.geeksforgeeks.org/umap-uniform-manifold-approximation-and-projection/

In [ ]:
#pip install umap-learn
from sklearn.preprocessing import StandardScaler
import umap

scaler = StandardScaler()
digits_umap = scaler.fit_transform(digits.data)

# Apply UMAP
reducer = umap.UMAP()
embedding = reducer.fit_transform(digits_umap)
plt.scatter(embedding[:, 0], embedding[:, 1], c=digits.target, cmap='Spectral', s=5)
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.show()

### Summary of Clustering Methods

### Ref
* Naive Bayes, Clearly Explained: https://www.youtube.com/watch?v=O2L2Uv9pdDA 
* Gaussian Naive Bayes, Clearly Explained: https://www.youtube.com/watch?v=H3EjCKtlVog 
* A Step-by-Step Explanation of Principal Component Analysis (PCA): https://builtin.com/data-science/step-step-explanation-principal-component-analysis 
* t-SNE, Clearly Explained: https://www.youtube.com/watch?v=NEaUSP4YerM        
* Naive Bayes Classifier: https://towardsdatascience.com/naive-bayes-classifier-81d512f50a7c